In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_model_optimization as tfmot


In [ ]:
input_shape = [20]
x_train = np.random.randn(1, 20).astype(np.float32)
y_train = tf.keras.utils.to_categorical(np.random.randn(1), num_classes=20)

def setup_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(20, input_shape=input_shape),
        tf.keras.layers.Flatten()
    ])
    return model

def setup_pretrained_weights():
    model = setup_model()
    model.compile(
        loss=tf.keras.losses.categorical_crossentropy,
        optimizer='adam',
        metrics=['accuracy']
    )
    model.fit(x_train, y_train)
    pretrained_weights = './result/model_without_quantization.tf'
    model.save_weights(pretrained_weights)
    return pretrained_weights

def set_pretrained_model():
    model = setup_model()
    pretrained_weights = setup_pretrained_weights()
    model.load_weights(pretrained_weights)
    return model

setup_model()
pretrained_weights = setup_pretrained_weights()


## 量化整个模型

1. 尝试量化某些对模型性能影响不大的神经层
2. 与从头开始训练相比，使用量化感知训练进行微调的效果一般更好 

In [ ]:
base_model = setup_model()
base_model.load_weights(pretrained_weights) # optional but recommended for model accuracy

quant_aware_model = tfmot.quantization.keras.quantize_model(base_model)
quant_aware_model.summary()